In [88]:
import urllib
from bs4 import BeautifulSoup
import requests
import numpy
import os
import re
import time
import json
import datetime
import unicodedata

In [3]:
url_general = 'https://maldita.es/malditobulo/1'
html_general = requests.get(url_general).text
soup_general = BeautifulSoup(html_general, 'html.parser')

In [4]:
clase_nav_bar = 'bg-white text-gray-800 px-3 py-2 text-lg border-gray-400 font-bold'

pags_link = soup_general.find('span', attrs={'class' : clase_nav_bar})('a')[0].get('href', None)
num_pags = int(pags_link.split('/')[-1])
print(num_pags)

565


In [121]:
def replace_characters(text):
    
    text_aux = ''.join(c for c in unicodedata.normalize('NFD', text.strip())
                  if unicodedata.category(c) != 'Mn')
    return re.sub(r'[^\w\s]', '', text_aux.lower().replace(u'\xa0', u' ')).split(" ")

In [133]:
bulos = {}

clase_new = 'section-card-headline'
cont = 0
for i in range(1, num_pags+1):
    url = 'https://maldita.es/malditobulo/'+str(i)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    for div in soup.find_all('div',attrs={'class' : clase_new}):
        try:
            a = div('a')[0]
            title = a.text
            link = a.get('href', None)
            words = dict()
            
            # Obtener la fecha de la noticia
            if "https://maldita.es/malditobulo/" in link:
                date = link.replace("https://maldita.es/malditobulo/", "")[0:8]
            elif "https://migracion.maldita.es/" in link:
                date = link.replace("https://migracion.maldita.es/", "")[0:8]
            else:
                date = link.replace("https://alimentacion.maldita.es/mitos-bulos/", "")[0:8]
            
            formated_date =  datetime.datetime.strptime(date, '%Y%m%d').strftime('%d/%m/%Y')
            
            html_new = requests.get("https://maldita.es/malditobulo/20211209/video-erupcion-volcan-semeru-indonesia-montaje/", allow_redirects=False).text

            soup_new = BeautifulSoup(html_new, 'html.parser')

            title_list = replace_characters(title.lower())


            for w in title_list:
                if w != '':
                    if w in words:
                        words[w] += 1
                    else:
                        words[w] = 1

                    
            new_text = soup_new.find('div',attrs={'id' : "article-text"})            
            
            text_list = new_text('p') + new_text('li')

            
            for text in text_list:
                if(not text.get_text().startswith("Una publicación compartida de")):
                    for w in replace_characters(text.get_text()):
                        if w != '':
                            if w in words:
                                words[w] += 1
                            else:
                                words[w] = 1


            bulos[cont] = {"titulo" : title, "link" : link, "date" : formated_date, "words_count" : words}
        except:
            print("No se ha podido obtener la informacion de la noticia", i)
        cont += 1
    break

In [134]:
bulos

{0: {'titulo': 'Cuidado con este SMS que dice que debes “activar un nuevo sistema de seguridad web”: no es de CaixaBank o Bankia, es ‘phishing’',
  'link': 'https://maldita.es/malditobulo/20211217/SMS-activar-nuevo-sistema-seguridad-web-caixabank-bankia-phishing/',
  'date': '17/12/2021',
  'words_count': {'cuidado': 1,
   'con': 7,
   'este': 5,
   'sms': 1,
   'que': 32,
   'dice': 1,
   'debes': 1,
   'activar': 1,
   'un': 10,
   'nuevo': 1,
   'sistema': 1,
   'de': 29,
   'seguridad': 1,
   'web': 1,
   'no': 4,
   'es': 12,
   'caixabank': 1,
   'o': 1,
   'bankia': 1,
   'phishing': 1,
   'mas': 1,
   '4400': 1,
   'retuits': 1,
   'y': 12,
   '17500': 1,
   'me': 1,
   'gusta': 1,
   'alcanza': 1,
   'una': 5,
   'publicacion': 1,
   'video': 9,
   'muestra': 3,
   'supuestamente': 2,
   'la': 32,
   'erupcion': 13,
   'del': 8,
   'volcan': 11,
   'semeru': 6,
   'en': 36,
   'indonesia': 8,
   'grabacion': 3,
   'se': 15,
   'ha': 4,
   'compartido': 2,
   'mensajes': 1,
   

In [9]:
len(bulos)

5635

In [10]:
with open('maldita.json', 'w') as fp:
    json.dump(bulos, fp)